In [188]:
from IPython.display import display, HTML, Image, clear_output
import time
import random
from ipywidgets import widgets
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import json

In [189]:
#to send data to google forms
def send_to_google_form(data_dict, form_url):
    
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok
  

In [190]:
#all photos used in the test are defined here
pic1 = Image("1_12v9.png", width = 500)
pic2 = Image("2_9v12.png", width = 500)

pic3 = Image("2_18v21.png", width = 500)
pic4 = Image("1_21v18.png", width = 500)

pic5 = Image("1_20v15.png", width = 500)
pic6 = Image("2_15v20.png", width = 500)

pic7 = Image("2_16v18.png", width = 500)
pic8 = Image("1_18v16.png", width = 500)

pic9 = Image("1_20v18.png", width = 500)
pic10 = Image("2_18v20.png", width = 500)

pic11 = Image("1_10v9.png", width = 500)
pic12 = Image("2_9v10.png", width = 500)

pic13 = Image("1_14v12.png", width = 500)
pic14 = Image("2_12v14.png", width = 500)

pic15 = Image("1_16v12.png", width = 500)
pic16 = Image("2_12v16.png", width = 500)

test_photos = ["1_12v9.png", "2_9v12.png", "2_18v21.png", "1_21v18.png", "1_20v15.png", "2_15v20.png", "2_16v18.png", "1_18v16.png", "1_20v18.png", "2_18v20.png", "1_10v9.png", "2_9v10.png", "1_14v12.png", "2_12v14.png", "1_16v12.png", "2_12v16.png"]

#this dictionary basically tells us which ratio category a particular photo belongs to
ratios = {
    'a': ["1_12v9.png", "2_9v12.png", "1_20v15.png", "2_15v20.png", "1_16v12.png", "2_12v16.png"],
    'b': ["2_18v21.png", "1_21v18.png", "1_14v12.png", "2_12v14.png"],
    'c': ["2_16v18.png", "1_18v16.png"],
    'd': ["1_20v18.png", "2_18v20.png", "1_10v9.png", "2_9v10.png"]  
}

#this is to use later on to see which ratio category a particular photo belongs to
def find_group(photo):
    for group, photos in ratios.items():
        if photo in photos:
            return group

In [191]:
#this enables text on the screen to appear gradually to make it easier for participants to follow instructions
def display_gradually(text, delay=0.05):
    for char in text:
        print(char, end='', flush=True)
        time.sleep(delay)
    print()
    return

In [192]:
#this basically introduces the user to what the test is about, and what they need to do
def introduction(): 
    intro_out = "This is the Approximate Number Sense (ANS) test. It basically tests how well you can approximate numerical quantities, without the use of language!"
    display_gradually(intro_out)
    time.sleep(1)

    intro_out_2 = "To test your ANS, we will show you a series of photos with two groups of dots on them - one will contain blue dots and the other one orange dots."
    display_gradually(intro_out_2)
    time.sleep(1)

    intro_out_3 = "They will basically look like this:"
    display_gradually(intro_out_3)

    display(pic1)
    time.sleep(3)

    intro_out_4 = "Each photo will be shown to you only briefly, and you will have 3 seconds to give us your answer by clicking either on a button that says blue (if you think there are more blue dots) or on a button that says orange (if you think there are more orange dots)."
    display_gradually(intro_out_4) 
    time.sleep(1)

    intro_out_5 = "Before we start, we will just ask you a few questions so that we know a little more about you!"
    display_gradually(intro_out_5)
    time.sleep(3)

    return


In [193]:
#this function shuffles the list of all test photos and creates a list of correct answers for the shuffled list
correct_answer_list = []

def get_photo_and_ans_list():
    
    random.shuffle(test_photos) #so that each participant receives a different order of test photos
    
    
    len_test = len(test_photos)
    
    for i in range (len_test):
        individual_correct_ans = test_photos[i][0]
        correct_answer_list.append(individual_correct_ans)

    return test_photos, correct_answer_list


In [194]:
#this function creates a blue response button

custom_style_blue = """
<style>
.jupyter-widgets.widget-button_blue {
    background-color: blue;
    color: white;
}
 </style>
"""
    
display(HTML(custom_style_blue))
    
#here the button is created
button_blue = widgets.Button(description='Blue')
    
button_blue.add_class('widget-button_blue')
    
#here the button is stylised 
button_blue.style.button_color = 'blue'
button_blue.style.color = 'white'
    
#display(button_blue)

In [195]:
#this function creates an orange response button

custom_style_orange = """
<style>
.jupyter-widgets.widget-button_orange {
    background-color: orange;
    color: white;
}
</style>
"""
    
display(HTML(custom_style_orange))
    
#here the button is created
button_orange = widgets.Button(description='Orange')
    
button_orange.add_class('widget-button_orange')
    
#here the button is stylised 
button_orange.style.button_color = 'orange'
button_orange.style.color = 'white'

#display(button_orange)

In [196]:
#this part of the code deals with widgets

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return event_info

In [197]:
#to store user responses
user_responses = []

In [198]:
# the test consists of showing the 16 defined photos four times in random order
#this function defines one out of these four rounds
def choose_winner():
    
    get_photo_and_ans_list()

    button_blue.on_click(register_btn_event)
    button_orange.on_click(register_btn_event)
    
    for i in range (len(test_photos)):
            pic = Image(test_photos[i], width = 500)
            display(pic)
            time.sleep(0.75)
            clear_output(wait = False)
            panel = widgets.HBox([button_blue, button_orange])
    
            display(panel)
            
            result = wait_for_event(timeout=3)
    
            if result['description'] == 'Blue':
                print("Blue button was pressed")
            elif result['description'] == 'Orange':
                print("Orange button was pressed")
            elif result['description'] == '':
                print("No button was pressed! Answer more quickly")
    
            if result['description']=='Blue':
                user_responses.append(1)
            elif result['description']=='Orange':
                user_responses.append(2)
            elif result['description'] == '':
                user_responses.append(0)
    
            time.sleep(1)
            clear_output(wait = False)

    #print(correct_answer_list)
    #print(user_responses)

    return user_responses

In [199]:
#a combination of all previously defined functions that runs the whole ANS test
def whole_ANS_test():
    introduction()
    clear_output(wait = False)
    
    user_name = input("What's your name?")
    
    user_age = input("And how old are you? Please enter a number!")

    while not user_age.isdigit():
        print("Invalid input. Please enter a number.")
        user_age = input("How old are you? ")

        
    gender_categories = ["m", "f", "nb", "o", "pns"]
    user_gender = input("What's your gender? Press f for female, m for male, nb for non-binary, o for other and pns if you would rather not say!")
    if user_gender not in gender_categories:
        print("Please format your response differently!")
        user_gender = input("What's your gender? Press f for female, m for male, nb for non-binary, o for other and pns if you would rather not say!")
            

    print("Great, now we know you a bit better! Now we can start the test!")

    time.sleep(3)

    clear_output(wait = False)
    
    ratio_list = []
    
    for i in range (4):
        choose_winner()
        for i in range (len(test_photos)):
            new = find_group(test_photos[i])
            ratio_list.append(new)

    score = 0
    for i in range (len(user_responses)):
        if int(user_responses[i])== int(correct_answer_list[i]):
            score = score +1
    print("You scored", score, "out of 64!")

    answer_string = ''.join(correct_answer_list)
    response_string = ''.join(map(str, user_responses))
    ratio_string = ''.join(ratio_list)

    data_dict = {

    'name': user_name,
    
    'age': user_age,
    
    'gender': user_gender,

    'score': score,

    'correct answer list': answer_string,

    'user responses': response_string,

    'photo list': ratio_string
    }

    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSewUYQ8u7q1iBiWtQS6lNBnKba5CjpLGqfjHoUkn1menM7TbQ/viewform?usp=sf_link'
    send_to_google_form(data_dict, form_url)
    
    return

In [200]:
whole_ANS_test()